In [ ]:
import cv2
import numpy as np
import os
import mediapipe as mp
import json

# --- ★変更点: 描画するパーツを選択するフラグ ---
# Trueに設定したパーツのランドマークが描画・保存されます
DRAW_FACE = False
DRAW_HANDS = True
DRAW_POSE = False

# 設定
DATA_PATH = os.path.join(os.getcwd(), 'MP_Data_JSON')
actions = np.array(['ageru', 'understand', 'annsinnsuru' , 'heavy'])  #クラスの部分。必要に応じて増やしていく。
no_videos = 10  #入力する動画数(指定したファイルのパス)
sequence_length = 30  #指定するフレーム数

# MediaPipe Holisticモデルの準備
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def multiple_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    """
    指定されたフラグに基づいて、顔、手、ポーズのランドマークを描画する関数
    """
    # 顔のランドマークを描画 (DRAW_FACEがTrueの場合)
    if DRAW_FACE and results.face_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.face_landmarks,
            mp_holistic.FACEMESH_TESSELATION, # 顔のメッシュ
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(200, 200, 200), thickness=1, circle_radius=1) # メッシュを薄い灰色で描画
        )
        mp_drawing.draw_landmarks(
            image,
            results.face_landmarks,
            mp_holistic.FACEMESH_CONTOURS, # 顔の輪郭
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=1, circle_radius=1) # 輪郭を少し濃い灰色で描画
        )

    # 手のランドマークを描画 (DRAW_HANDSがTrueの場合)
    if DRAW_HANDS:
        # 左手
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2), # 点を赤色で描画
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 100, 100), thickness=2) # 線を薄い赤色で描画
            )
        # 右手
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2), # 点を青色で描画
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(100, 100, 255), thickness=2) # 線を薄い青色で描画
            )

    # 姿勢のランドマークを描画 (DRAW_POSEがTrueの場合)
    if DRAW_POSE and results.pose_landmarks:
        # この部分は元のコードから削除されていますが、必要であれば
        # DESIRED_POSE_LANDMARKSを定義した上で、元の描画ロジックをここに記述してください。
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(51, 255, 51), thickness=2)
        )


def extract_keypoints(results):
    """
    指定されたフラグに基づいて、顔、手、ポーズのキーポイントを抽出する関数
    """
    keypoints_data = {}

    # 顔のランドマークを抽出 (DRAW_FACEがTrueの場合)
    if DRAW_FACE:
        if results.face_landmarks:
            face_landmarks_data = [{"id": i, "x": res.x, "y": res.y, "z": res.z} for i, res in enumerate(results.face_landmarks.landmark)]
            keypoints_data["face"] = face_landmarks_data
        else:
            keypoints_data["face"] = []

    # 左手のランドマークを抽出 (DRAW_HANDSがTrueの場合)
    if DRAW_HANDS:
        if results.left_hand_landmarks:
            left_hand_landmarks_data = [{"id": i, "x": res.x, "y": res.y, "z": res.z} for i, res in enumerate(results.left_hand_landmarks.landmark)]
            keypoints_data["left_hand"] = left_hand_landmarks_data
        else:
            keypoints_data["left_hand"] = []

    # 右手のランドマークを抽出 (DRAW_HANDSがTrueの場合)
    if DRAW_HANDS:
        if results.right_hand_landmarks:
            right_hand_landmarks_data = [{"id": i, "x": res.x, "y": res.y, "z": res.z} for i, res in enumerate(results.right_hand_landmarks.landmark)]
            keypoints_data["right_hand"] = right_hand_landmarks_data
        else:
            keypoints_data["right_hand"] = []

    # 姿勢のランドマークを抽出 (DRAW_POSEがTrueの場合)
    if DRAW_POSE:
        if results.pose_landmarks:
            pose_landmarks_data = [{"id": i, "x": res.x, "y": res.y, "z": res.z, "visibility": res.visibility} for i, res in enumerate(results.pose_landmarks.landmark)]
            keypoints_data["pose"] = pose_landmarks_data
        else:
            keypoints_data["pose"] = []

    return keypoints_data

# ★★★ ここからが新しく追加・修正した部分です ★★★

def resample_keypoints(keypoints_list, target_frames):
    """
    キーポイントデータのリストを、線形補間または間引きによって
    指定したターゲットフレーム数にリサンプリングする。
    """
    actual_frames = len(keypoints_list)
    if actual_frames == target_frames:
        return keypoints_list
    
    # 元のフレームのインデックスと新しいフレームのインデックスを生成
    original_indices = np.linspace(0, actual_frames - 1, actual_frames)
    target_indices = np.linspace(0, actual_frames - 1, target_frames)
    
    resampled_list = []
    
    for t_idx in target_indices:
        # t_idxに最も近い元のフレームのインデックスを探す
        p1_idx = int(np.floor(t_idx))
        p2_idx = int(np.ceil(t_idx))

        if p1_idx == p2_idx: # 補間不要（間引きの場合など）
            resampled_list.append(keypoints_list[p1_idx])
            continue
        
        # 線形補間の比率を計算
        ratio = t_idx - p1_idx
        if ratio < 1e-6: # ほぼp1と同じ位置
             resampled_list.append(keypoints_list[p1_idx])
             continue

        # 補間元のフレームデータを取得
        kp1 = keypoints_list[p1_idx]
        kp2 = keypoints_list[p2_idx]
        
        # 新しい補間フレーム用のデータ構造を作成
        interpolated_kp = {}
        
        # 各部位（'face', 'left_hand'など）についてループ
        all_keys = set(kp1.keys()) | set(kp2.keys())
        for part in all_keys:
            part_kp1 = {lm['id']: lm for lm in kp1.get(part, [])}
            part_kp2 = {lm['id']: lm for lm in kp2.get(part, [])}
            
            interpolated_part = []
            
            # 両方のフレームに存在するランドマークIDでループ
            common_ids = set(part_kp1.keys()) & set(part_kp2.keys())
            for lm_id in sorted(list(common_ids)):
                lm1 = part_kp1[lm_id]
                lm2 = part_kp2[lm_id]
                
                new_lm = {'id': lm_id}
                # 各座標を線形補間
                for coord in ['x', 'y', 'z']:
                    if coord in lm1 and coord in lm2:
                        new_lm[coord] = lm1[coord] * (1 - ratio) + lm2[coord] * ratio
                
                # visibilityもあれば補間（poseのみ）
                if 'visibility' in lm1 and 'visibility' in lm2:
                    new_lm['visibility'] = lm1['visibility'] * (1 - ratio) + lm2['visibility'] * ratio
                
                interpolated_part.append(new_lm)
            
            interpolated_kp[part] = interpolated_part
            
        resampled_list.append(interpolated_kp)
        
    return resampled_list

# データの収集
cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
cv2.resizeWindow('OpenCV Feed', (1280, 720))

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        if not os.path.exists(action_path):
            os.makedirs(action_path)

        for video_num in range(1, no_videos + 1):
            video_file_path = os.path.join(r"D:\\卒研手話\\video\\hikensya5", action + str(video_num) + '.mp4')
            if not os.path.exists(video_file_path):
                print(f"Warning: Video file not found, skipping: {video_file_path}")
                continue

            cap = cv2.VideoCapture(video_file_path)

            # 1. まず動画の全フレームからキーポイントを抽出してリストに保存
            all_keypoints = []
            print(f'Processing video: {video_file_path}')
            while True:
                ret, frame = cap.read()
                if not ret:
                    break # 動画の終わり
                
                image, results = multiple_detection(frame, holistic_model)
                draw_styled_landmarks(image, results)
                
                # 処理中のフレームを表示
                cv2.putText(image, f'Extracting from {action} - Video {video_num}',
                            (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                all_keypoints.append(keypoints)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            cap.release()
            
            actual_frame_count = len(all_keypoints)
            if actual_frame_count == 0:
                print(f"Warning: No frames extracted from {video_file_path}. Skipping.")
                continue
                
            print(f'  -> Extracted {actual_frame_count} frames. Resampling to {sequence_length} frames.')
            
            # 2. フレーム数をsequence_lengthにリサンプリング（補間 or 間引き）
            resampled_keypoints = resample_keypoints(all_keypoints, sequence_length)

            # 3. リサンプリング後のキーポイントをJSONファイルに保存
            video_data_save_path = os.path.join(action_path, str(video_num))
            if not os.path.exists(video_data_save_path):
                os.makedirs(video_data_save_path)
            
            for frame_num, keypoints in enumerate(resampled_keypoints):
                json_path = os.path.join(video_data_save_path, f'{frame_num}.json')
                with open(json_path, 'w') as f:
                    json.dump(keypoints, f, indent=4)
            
            print(f'  -> Saved {len(resampled_keypoints)} frames to {video_data_save_path}')

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

# ★★★ ここまでが新しく追加・修正した部分です ★★★